<a href="https://colab.research.google.com/github/andrkech/GENERATIVE-METHODS-IN-GENOMICS/blob/main/Variant_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd

In [ ]:
def introduce_variant(seq, position, ref_base, alt_base):
  seq_list = list(seq)
  seq_list[position] = alt_base
  return "".join(seq_list)

In [ ]:
csv_file_oath = "/content/..."
pd.read_csv(csv_file_path)
df = pd.read_csv(csv_file_path)

In [ ]:
try:
  line_number = int(input("Choose a variant from the CSV file"))

  row = df.iloc[line_number]

  variant_info = {
                  "SampleID": selected_row["SampleID"],
                  "Gene": selected_row["Gene.refGene"],
                  "Chromosome": selected_row["CHROM"],
                  "Position": selected_row["POS"],
                  "RefBase": selected_row["REF"],
                  "AltBase": selected_row["ALT"]
                  }

'''
for key, value in variant_info.items():
  print(f"{key}: {value}")
'''
except (ValueError, IndexError):
  print("Invalid line number")